In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:rgb(0,200,100);">1. 패키지 </span>

In [5]:
# python-dotenv, langchain(1.2.0),  langchain-openai,  langchain-pinecone,
# pandas,  langchain-community, docx2txt,  langchain-text_splitters,  langchain_ollama

# <span style="color:rgb(0,200,100);">2. 환경설정(환경변수, 시스템파라미터변수) </span>

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_LLM_MODEL = "gpt-4o-mini"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-large" # 차원수 3072

PINECONE_INDEX_NAME = "better-rag-index"
PINECONE_INDEX_DEMENSION = 3072
PINECONE_INDEX_METRIC = "cosine"
PINECONE_INDEX_REGION = "us-east-1"
PINECONE_INDEX_CLOUD = "aws"

# <span style="color:rgb(0,200,100);">3. 문서를 chunk로 분할하기 </span>

In [7]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./data/소득세법_with_markdown.docx')
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200, 
    separators=["\n\n", "\n", " ", ""]
)
# documents = loader.load_and_split(text_splitter)
documents = text_splitter.split_documents(document)
print(f"총 {len(documents)}개 청크 생성")

총 194개 청크 생성


# <span style="color:rgb(0,200,100);">4. metadata 추가하기 </span>
- 청크 내용의 카테고리, 청크내용의 title, 조항

In [13]:
import re
def remove_special_chars(text:str) -> str:
    "특수문자 및 \n제거(:는 그대로)"
    # \n제거
    text = text.replace("\n", " ")
    # 한글, 영문, 숫자, 공백, 마침표, 콤마, :만 남기고 전부 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s,\.:]', '', text)
    # 불필요한 중복 공백을 제거
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned
# 사용예시
content = "**소득세 납세 의무 **: \n\n이 법 또는 「법인세법」에 따라 소득에 대한 소득세를 부과"
print(remove_special_chars(content))

소득세 납세 의무 : 이 법 또는 법인세법에 따라 소득에 대한 소득세를 부과


In [19]:
# 제목을 추출하는 함수(exaone3.5) : powershell이나 cmd창에서 ollama pull exaone3.5:2.4b
from langchain_ollama import ChatOllama
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extractor_llm = ChatOllama(
        model = "exaone3.5:2.4b",
        temperature = 0.1,
        num_predict = 30 # 최대 토큰 30토큰까지만 출력
    )
    prompt = f"""다음 소득세법 조문의 핵심 제목을 30토큰 이내로 완벽하게 말이 되도록 추출해 주세요.
    중간에 말이 끊기면 안 되요.
    예 : "소득세 납세의미 범위 : 공동사업자별 소득과세, 상속과세, 증여자"
    조문 : {content}"""
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return remove_special_chars(title)
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 원천징수하는 자가 법인인 경우: 그 법인의 본점 또는 주사무소의 소재지
"""
print(extract_title_with_llm(content))

소득세 원천징수 기준: 거주자비거주자별 소재지 규정 거주지사업장 소재지 우선 적용


In [24]:
def categorize_content(content):
    "내용 카테고리 분류"
    # 쳇 gpt가 meta 데이터에 쓴다고 하니 만들어 준 카테고리
    categories = {
    '납세의무': ['납세의무', '거주자', '비거주자', '원천징수', '공동사업자', '상속인', '증여자', '신탁재산'],
    '세율계산': ['세율', '과세표준', '산출세액', '결정세액', '종합소득', '퇴직소득'],
    '근로소득': ['근로소득', '총급여', '급여', '연봉', '임금', '퇴직소득'],
    '사업소득': ['사업소득', '공동사업', '주택임대소득', '부업소득'],
    '이자배당': ['이자소득', '배당소득', '예금이자', '채권', '의제배당'],
    '양도소득': ['양도소득', '자산양도', '부동산양도', '주식양도'],
    '연금소득': ['연금소득', '공적연금', '사적연금', '연금보험'],
    '기타소득': ['기타소득', '상금', '보상금', '발명보상금', '종교인소득'],
    '공제감면': ['공제', '소득공제', '세액공제', '기본공제', '감면'],
    '비과세': ['비과세', '면제', '복무급여', '실업급여', '출산휴가급여', '장학금'],
    '신고납부': ['신고', '납부', '납세지', '신고기한', '원천징수'],
    '과세기간': ['과세기간', '과세연도', '사업연도'],
    '과세소득구분': ['종합소득', '퇴직소득', '양도소득', '금융투자소득'],
    }
    result = [] # 각 카테고리 추가
    for category, keywords in categories.items():
        #print(category, keywords)
        #print(any([keyword in content for keyword in keywords]))
        if any([keyword in content for keyword in keywords]):
            result.append(category)
    if not result:
        result.append("기타")
    return result
categorize_content(content)

['납세의무', '신고납부']

- 개선된 카테고리 분류방법 : 키워드의 중요도에 따라 가중치를 부여하고, 가중치가 높은 순으로 카테고리 반환

In [25]:
def get_category():
    return {
        '납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },

        '세율계산': {
            '세율': 3, '소득세': 3, '과세표준': 3, '산출세액': 3, '세액': 3,
            '결정세액': 2, '세액계산': 2, '기본세율': 2, '세율적용': 2,
            '누진세율': 2, '종합소득세': 2
        },

        '근로소득': {
            '근로소득': 3, '총급여': 3, '급여': 3, '연봉': 3, '임금': 3,
            '근로소득금액': 2, '총급여액': 2, '상여': 2, '수당': 2,
            '봉급': 2, '직장인': 2
        },

        '사업소득': {
            '사업소득': 3, '총수입금액': 3, '필요경비': 3,
            '사업자': 2, '사업소득금액': 2, '결손금': 2, '이월결손금': 2,
            '주택임대소득': 2, '공동사업': 2
        },

        '이자배당': {
            '이자소득': 3, '배당소득': 3, '예금이자': 2, '채권': 2,
            '의제배당': 2, '배당세액공제': 2, '분리과세이자소득': 2,
            '분리과세배당소득': 2
        },

        '양도소득': {
            '양도소득': 3, '자산양도': 2, '부동산양도': 2, '주식양도': 2,
            '양도차익': 2, '취득가액': 2, '양도가액': 2, '양도소득금액': 2
        },

        '연금소득': {
            '연금소득': 3, '연금계좌': 3, '연금저축': 3,
            '퇴직연금': 2, '공적연금': 2, '사적연금': 2, '연금보험': 2,
            '연금수령': 2
        },

        '기타소득': {
            '기타소득': 3, '가상자산': 3, '가상자산소득': 2,
            '상금': 2, '보상금': 2, '종교인소득': 2, '원고료': 2,
            '복권': 1, '당첨금': 1, '발명보상금': 1
        },

        '공제감면': {
            '공제': 3, '소득공제': 3, '세액공제': 3,
            '기본공제': 2, '인적공제': 2, '특별공제': 2, '추가공제': 2,
            '근로소득공제': 2, '연금소득공제': 2, '퇴직소득공제': 2,
            '연금계좌세액공제': 2, '감면': 2
        },

        '비과세': {
            '비과세': 3, '비과세소득': 3, '면제': 2, '세액감면': 2,
            '소득세면제': 2, '복무급여': 1, '실업급여': 1, 
            '출산휴가급여': 1, '장학금': 1
        },

        '신고납부': {
            '신고': 3, '확정신고': 3, '과세표준확정신고': 3, 
            '납부': 3, '중간예납': 2, '가산세': 2,
            '신고기한': 2, '납부기한': 2, '납세지': 2
        },

        '과세기간': {
            '과세기간': 3, '과세연도': 3, '사업연도': 2, 
            '과세기간종료일': 2
        },

        '과세방식': {
            '종합과세': 3, '분리과세': 3, '합산과세': 2, 
            '종합소득과세표준': 2, '분리과세소득': 2, 
            '금융투자소득': 2
        },

        '장부기장': {
            '장부': 3, '복식부기': 3, '간편장부': 2, '기장': 2,
            '장부기록': 2, '증명서류': 2, '기장세액공제': 2
        }
    }

In [30]:
content = """1. 원천징수하는 자가 거주자인 경우: 그 거주자의 주된 사업장 소재지. 다만, 주된 사업장 외의 사업장에서 원천징수를 하는 경우에는 그 사업장의 소재지, 사업장이 없는 경우에는 그 거주자의 주소지 또는 거소지로 한다.
2. 원천징수하는 자가 비거주자인 경우: 그 비거주자의 주된 국내사업장 소재지. 다만, 주된 국내사업장 외의 국내사업장에서 원천징수를 하는 경우에는 그 국내사업장의 소재지, 국내사업장이 없는 경우에는 그 비거주자의 거류지(居留地) 또는 체류지로 한다.
3. 종합과세 분리과세 합산과세 종합소득과세표준 분리과세소득 금융투자소득
"""

In [31]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            if keyword in content:
                score += weight
        if score > 0:
            category_scores[category] = score
    print(category_scores)
categorize_content(content)

{'납세의무': 8, '세율계산': 3, '과세방식': 14}


In [ ]:
'납세의무': {
            '납세의무': 3, '거주자': 3, '비거주자': 3, '납세의무자': 3, 
            '원천징수': 2, '원천징수의무자': 2, '공동사업자': 2, 
            '상속인': 2, '증여자': 2, '신탁재산': 2
        },